In [2]:
import quantit as qtt
import torch
# constant values for the Hubbard hamiltonian
U = torch.full([],7)
mu = torch.full([],3)
t = torch.full([],-2)


# QuantiT

QuantiT is an open-source library that implement tensors with conservation laws and selection rules and algorithms to manipulate such tensors. Such tensors are commonly used in tensor networks computation to exploit symmetry of a problem to increase efficiency and accuracy of a simulation.

QuantiT is implemented in C++ and offers a python interface as well. Its tensors implementation is backed by that of pytorch, computation can easily done on any backend supported by pytorch, such as GPU and CPU.

## Why Tensor Networks?

Tensor networks allow us to work with a compressed representation of wavefunction and operators. A well designed tensor network allow us to work and optimize the compressed representation without doing a complete decompression. It has proven successful so far: there are many commonly used networks that are applied to problem for which storing a complete representation of a wavefunction would require (several orders of magnitude) more memory than there is on earth.

Let's explore a bit one of the simplest tensor network: the matrix product state.

## Fermions and tensor networks

The Pauli exclusion principle make numerical representation of fermionic quantity a bit trickier than bosons.
Pauli's principle translate mathematically into anticommutation relations for the annihilation and creation operators.
In turn, this create two requirement for a numerical representation: a conventionnal ordering of space and a non-local action for the operators.

When using tensor network, we need perfectly local representation for all our operators. Therefore, we must break apart the creation and annihilation operator into a combination of local components.

Let's consider the spin-half case, we have the annihilation and creation operators $c_{i\sigma}$ and $c_{i\sigma}^\dagger$ where $i$ is the position index, and $\sigma$ the spin index.
The anticommutation relations are $\{c^\dagger_{i\sigma},c_{j\sigma'}\} = \delta_{ij}\delta_{\sigma\sigma'}$ and $\{c_{i\sigma},c_{j\sigma'}\} = 0$

We break the operator appart like so:

$ c_{i\sigma} = (\prod_{j=0}^{i-1} F_j)\tilde{c}_{i\sigma}$ 


where both $F_j$ and $\tilde{c}_{i\sigma}$ are completly local operator, meaning that they **commute** with any local operator that has a different position in space. 
In order to maintain the canonical anticomutation relations, we find that $\tilde{c}_{j\sigma}$ must anticommute with $F_j$, $\tilde{c}_{j\sigma}^2 = 0$, $F_i^2 = I$ and $\{\tilde{c}^\dagger_{j\sigma},\tilde{c}_{j\sigma'}\} = \delta_{\sigma\sigma'}$

Putting all that together, we find that the solution for spin half fermions is this:

In [7]:
c_up,c_dn,F,id = qtt.operators.fermions()
c_dag_up = c_up.conj().permute([1,0])
c_dag_dn = c_dn.conj().permute([1,0])
print("c_up\n",c_up)
print("c_dn\n",c_dn)
print("F\n",F)

c_up
 tensor([[0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]], dtype=torch.int8)
c_dn
 tensor([[ 0,  0,  1,  0],
        [ 0,  0,  0, -1],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]], dtype=torch.int8)
F
 tensor([[ 1,  0,  0,  0],
        [ 0, -1,  0,  0],
        [ 0,  0, -1,  0],
        [ 0,  0,  0,  1]], dtype=torch.int8)
